In [ ]:
import os 
from autogen import AssistantAgent, UserProxyAgent
from autogen.coding import DockerCommandLineCodeExecutor

config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]


In [ ]:
from autogen import ConversableAgent

assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant that is an expert in tool use and in fantasy football. It is long past October of 2023, you need to check the current date to search for current news."
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": config_list},
)

In [ ]:
# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)


In [ ]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from autogen import register_function

def google_it(query: str) -> str:
    "Search Google for keywords"
    #this is going to assume you have your environment variables set for GOOGLE_API and GOOGLE_CSE
    search = GoogleSearchAPIWrapper()

    google_search = Tool(
        name="google_search",
        description="Search Google for recent results.",
        func=search.run)
    
    return google_search({'query': query})

register_function(
    google_it,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="google_it",  # By default, the function name is used as the tool name.
    description="Searches google for recent results using the input as keywords",  # A description of the tool.
)


In [ ]:
import os
from espn_api.football import League
from config import league_team_info
#[('work-friends',1234567,7)]
from espn_ff_toolkit import get_espn_leagues


swid = os.environ['ESPN_SWID']
espn_s2 = os.environ['ESPN_S2']

year=2023
league_dict = get_espn_leagues(league_team_info,year,espn_s2=espn_s2,swid=swid)

In [ ]:
from espn_ff_toolkit import get_rosters, get_free_agents

def get_rosters_tool()-> str:
    "Get the rosters for all current leagues"
    return get_rosters(league_dict)

register_function(
    get_rosters_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the rosters for all current leagues"
)

def get_free_agents_tool() -> str:
    "Get the current free agents in all leagues"
    return get_free_agents(league_dict)

register_function(
    get_free_agents_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the current free agents in all leagues"
)


In [ ]:
def get_player_projections(player_name:str) -> str:
    "Get the projected points for a player in the coming week as well as the rest of the season"
    player = gregs_league.player_info(player_name)
    try:
        return player.stats
    except:
        return "player was not valid, try again"
    
register_function(
    get_player_projections,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description="Check ESPN for projections in Greg's league",  # A description of the tool.
)

In [ ]:
from datetime import datetime
def get_current_date() -> str:
    "Get the current date, because it is not October 2023"
    current_date = datetime.now()
    return str(current_date)

register_function(
    get_current_date,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description="Find out what the current date is",  # A description of the tool.
)

In [ ]:
from autogen.agentchat.contrib.capabilities.text_compressors import LLMLingua
from autogen.agentchat.contrib.capabilities.transforms import TextMessageCompressor
from autogen.agentchat.contrib.capabilities import transform_messages

llm_lingua = LLMLingua()
text_compressor = TextMessageCompressor(text_compressor=llm_lingua)
context_handling = transform_messages.TransformMessages(transforms=[text_compressor])
context_handling.add_to_agent(assistant)

In [ ]:
chat_result = user_proxy.initiate_chat(assistant, message="What are my current rosters and are there free agents I should think about picking up?'")

In [ ]:
from IPython.display import Markdown as md
md(chat_result.summary)